In [ ]:
import numpy as np
import pandas as pd
import pyterrier as pt
if not pt.started():
  pt.init()

In [ ]:
def get_webtrack_gov():
#     for k in ["trec-wt-2002", "trec-wt-2003", "trec-wt-2004"]:
    for k in ["trec-wt-2004"]:
        ds = pt.get_dataset(k)
        print(dir(ds))
#         for t in ["np", "td", "hp"]:
        for t in ["all"]:
            topics = ds.get_topics(t)
            indexref=ds.get_index
            qrels = ds.get_qrels(t)
            return (qrels, topics, indexref)

qrels, topics, indexref = get_webtrack_gov()

In [ ]:
dataset = pt.datasets.get_dataset("vaswani")
#dataset = pt.datasets.get_dataset("trec-wt-2012")
print(dataset)
indexref = dataset.get_index()
print(type(indexref))
topics = dataset.get_topics()
qrels = dataset.get_qrels()

In [ ]:
print(type(qrels))
print(len(topics))
print(len(qrels))

In [ ]:
#this ranker will make the candidate set of documents for each query
BM25 = pt.BatchRetrieve(indexref, controls = {"wmodel": "BM25"})

#these rankers we will use to re-rank the BM25 results
TF_IDF =  pt.BatchRetrieve(indexref, controls = {"wmodel": "TF_IDF"})
PL2 =  pt.BatchRetrieve(indexref, controls = {"wmodel": "PL2"})
DPH = pt.BatchRetrieve(indexref, controls = {"wmodel": "DPH"})
PL2F =  pt.BatchRetrieve(indexref, controls = {"wmodel": "PL2F"})

In [ ]:
feature_batch_retriever = pt.FeaturesBatchRetrieve(indexref, features=["WMODEL:TF_IDF", "WMODEL:PL2", "WMODEL:DPH", "WMODEL:BM25"]) 

In [ ]:
(BM25 %2).transform("world")

In [ ]:
(PL2).transform("world")

In [ ]:
train_topics, valid_topics, test_topics = np.split(topics, [int(.6*len(topics)), int(.8*len(topics))])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

ltr_model = fbr >> pt.pipelines.LTR_pipeline(RandomForestRegressor(n_estimators=400))
ltr_model.fit(train_topics, qrels)

In [ ]:
results = pt.pipelines.Experiment([TF_IDF, BM25, PL2, ltr_model], test_topics, qrels, ["map", "ndcg"], names=["TF-IDF", "BM25 algorithm", "PL2 Baseline", "LTR Random Forest"])
results

In [ ]:
import matplotlib.pyplot as plt  # plotting libraries

def run_experiment(trained_model, mop, start= 10, finish = 600, incrementer = 25, top_k_model = BM25, title="sample title"):
    k_list = []
    moe_list = []
    for k in range(start, finish, incrementer):
        efficient_pipeline = top_k_model % k >> trained_model
        results = pt.pipelines.Experiment([efficient_pipeline], test_topics, qrels, [mop], names=["model"])
        k_list.append(k)
        moe_list.append(results[mop].iloc[0])
        
        plt.plot(k_list, moe_list)
        plt.xlabel("K")
        plt.ylabel(mop)
        file_name = title.replace(" ", "-")
        plt.title(title)
        plt.savefig(file_name)
        
run_experiment(ltr_model, "ndcg", title = "K's Affect on NDCG in Learning to Rank")